# Preprocessing

In [36]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [20]:
easyshare = pd.read_stata('data/sharewX_rel8-0-0_easySHARE_stata/easySHARE_rel8-0-0.dta')

illness_before = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/illness_before_module_v01.dta")
illness_during = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/illness_during_module_v01.dta")
job = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/job_module_v01.dta")
life = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/life_module_v01.dta")
young_age = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/young_age_module_v01.dta")
yearly = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/yearly_module_v01.dta")


In [21]:
df = pd.merge(easyshare, life, on=['mergeid', 'wave'], how='left')
df = pd.merge(df, job, on=['mergeid'], how='left')

In [22]:
df_relevant = df[df.columns.drop(list(df.filter(regex='^euro')))]
df_relevant = df_relevant[df_relevant.columns.drop(list(df_relevant.filter(regex='^dn')))]
non_predictive_vars = [
    'mergeid',    # Used for merging records, no predictive power
    'hhid',       # Household identifier for tracking or grouping data
    'coupleid',   # Links records of individuals within a household
    'int_version',# Version of the questionnaire or interview format
    'int_year',   # Year the interview was conducted, structural rather than predictive
    'int_month',  # Month the interview was conducted, similar to int_year
    'country',    # Country code, used for stratification or adjustments
    'country_mod', # Modified country code, typically for data manipulation
    'wavepart', # Wave part, used for stratification or adjustments
    'recall_1',
    'recall_2',   
]
df_relevant = df_relevant[df_relevant.columns.drop(non_predictive_vars)]

In [23]:
def replace_dash_with_na(df):
    for column in df.columns:
        if df[column].dtype == 'category':
            # Replace entries containing '-' with NA
            df[column] = df[column].apply(lambda x: pd.NA if '-' in str(x) else x)
    return df

df_relevant = replace_dash_with_na(df_relevant)

In [24]:
na_counts = df_relevant.groupby('wave').apply(lambda x: x.isnull().sum())
# mean per number of abservation per wave
na_counts['mean']= na_counts.mean(axis=1)
na_counts['obs'] = df_relevant.groupby('wave').size()
na_counts['avg_mean'] = na_counts['mean']/ na_counts['obs']
na_counts['std'] = na_counts.std(axis=1)
na_counts = na_counts.sort_values(by='avg_mean', ascending=False)
df_sorted = df_relevant.sort_values(by=['wave'], ascending=[False])
# drop all except wave 7 
df_wave_7 = df_sorted[df_sorted['wave'] == 7]
df_wave_7 = df_wave_7.drop(columns=['wave'])
# df_most_recent_wave_per_mergeid = df_sorted.drop_duplicates(subset='mergeid', keep='first')

In [25]:
na_counts = df_wave_7.isna().sum()

na_counts_sorted = na_counts.sort_values(ascending=False)

na_counts_sorted

emi_pm2p5_w      77202
bmi2             77202
emi_pm10_w       77202
income_pct_w8    77202
income_pct_w6    77202
                 ...  
thinc_m              0
hhsize               0
partnerinhh          0
female               0
language             0
Length: 413, dtype: int64

In [26]:
for column in df_wave_7.columns:
    if df_wave_7[column].dtype == object:  # Check if the column data type is object
        # Try converting the column to numeric
        converted_column = pd.to_numeric(df_wave_7[column], errors='coerce')
        # Check if the conversion did not introduce any new NaNs (i.e., all NaNs in the original are NaNs in the converted)
        if converted_column.notna().equals(df_wave_7[column].notna()):
            df_wave_7[column] = converted_column

df_wave_7.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77202 entries, 22153 to 32326
Columns: 413 entries, language to yjob_tx_g30_w
dtypes: category(7), float32(309), float64(50), object(47)
memory usage: 149.2+ MB


In [27]:
columns_to_drop = na_counts[na_counts > 20000].index


df_dropped = df_wave_7.drop(columns=columns_to_drop)

shape_of_dataframe_full = df_wave_7.shape
shape_of_dataframe_dropped = df_dropped.shape

print(f"No. of samples: {shape_of_dataframe_full[0]}")
print(f"No. of columns (full): {shape_of_dataframe_full[1]}")
print(f"No. of columns (dropped): {shape_of_dataframe_dropped[1]}")

columns_to_drop

No. of samples: 77202
No. of columns (full): 413
No. of columns (dropped): 266


Index(['q34_re', 'isced1997_r', 'int_partner', 'age_partner', 'gender_partner',
       'ch001_', 'ch021_mod', 'ch007_hh', 'ch007_km', 'sp002_mod',
       ...
       'avgjob_conc_yearly_o3_mean', 'avgjob_conc_yearly_o3_median',
       'avgjob_conc_yearly_o3_w', 'avgjob_emissions_PM10_mean',
       'avgjob_emissions_PM10_median', 'avgjob_emissions_PM10_w',
       'avgjob_emissions_PM25_mean', 'avgjob_emissions_PM25_median',
       'avgjob_emissions_PM25_w', 'job_uncomfortable'],
      dtype='object', length=147)

In [28]:
na_after_dr = df_dropped.isna().sum().sort_values(ascending=False)


In [29]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77202 entries, 22153 to 32326
Columns: 266 entries, language to yjob_tx_g30_w
dtypes: category(7), float32(221), float64(15), object(23)
memory usage: 88.6+ MB


# Finally one hot encoding!

In [30]:
def process_categorical_columns(df):
    for column in df.columns:
        if column != 'sphus':
            if (df[column].dtype == 'category' or df[column].dtype == 'object') and df[column].nunique() < 100:
                dummies = pd.get_dummies(df[column], prefix=column, drop_first=True)
                    # Merge these dummy variables back to the original DataFrame
                df = pd.concat([df, dummies], axis=1)
                df = df.drop(columns=column)

    return df

In [31]:
df_processed = process_categorical_columns(df_dropped)

In [32]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77202 entries, 22153 to 32326
Columns: 730 entries, age to job_start_2017
dtypes: bool(490), float32(221), float64(15), object(4)
memory usage: 112.9+ MB


In [33]:
categorical = df_processed.select_dtypes(include='category').columns.tolist()
print(categorical)

[]


In [34]:
df_processed.to_csv('data/processed_data.csv', index=False)

# HB edits 

1) for some reason the neural net script gave different responses, so we'll need to adapt accordingly (so we treat this subsequent processing as a template)

2) we should move these processing steps to before the X / y, test/train/val split so we're acting on the original df which we then can just export out as a csv and load directly into our other scripts as we're working on them.

df = pd.read_csv('data/processed_data.csv')
df.info()

In [37]:
X = df.drop('sphus', axis=1) 
y = df['sphus'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [38]:
train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 70% of the entire data set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, random_state = 10117)

# test is now 15% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = test_ratio / (test_ratio + validation_ratio), random_state = 10117)

n_train = X_train.shape[0]
n_val = X_val.shape[0]
n_test = X_test.shape[0]
n = n_train + n_val + n_test

print((n_train / n), (n_val / n), (n_test / n)) 

0.7 0.14999878673169784 0.15000121326830215


In [39]:
# training it: first need to remove all NaNs / impute

# Check for NaNs in the training data
if X_train.isna().any().any() or y_train.isna().any().any():
    print("NaNs in train data")

# Check for infinities in the training data
if (X_train == np.inf).any().any() or (y_train == np.inf).any().any():
    print("Infinities in train data")

NaNs in train data


### Impute & Scale

In [40]:
# Select non-numeric columns (objects and booleans)
non_numeric_columns_train = X_train.select_dtypes(include=['object', 'bool']).columns
non_numeric_columns_test = X_test.select_dtypes(include=['object', 'bool']).columns
non_numeric_columns_val = X_val.select_dtypes(include=['object', 'bool']).columns


# Impute non-numeric columns with the string "missing"
X_train[non_numeric_columns_train] = X_train[non_numeric_columns_train].fillna("missing")
X_test[non_numeric_columns_test] = X_test[non_numeric_columns_test].fillna("missing")
X_val[non_numeric_columns_val] = X_val[non_numeric_columns_val].fillna("missing")


# Select numeric columns
numeric_columns_train = X_train.select_dtypes(include=[np.number]).columns
numeric_columns_test = X_test.select_dtypes(include=[np.number]).columns
numeric_columns_val = X_val.select_dtypes(include=[np.number]).columns

# Initialize the imputer for numeric columns
numeric_imputer = SimpleImputer(strategy='mean')

# Apply imputation to numeric columns in training data
X_train[numeric_columns_train] = numeric_imputer.fit_transform(X_train[numeric_columns_train])

# Apply imputation to numeric columns in testing data using the same imputer
X_test[numeric_columns_test] = numeric_imputer.transform(X_test[numeric_columns_test])
X_val[numeric_columns_test] = numeric_imputer.transform(X_val[numeric_columns_test])

# initialise
scaler = StandardScaler()

# Scale training data (fit and transform)
X_train_scaled = X_train.copy()
X_train_scaled[numeric_columns_train] = scaler.fit_transform(X_train[numeric_columns_train])

# Scale testing data (only transform)
X_test_scaled = X_test.copy()
X_test_scaled[numeric_columns_test] = scaler.transform(X_test[numeric_columns_test])

# Scale validation data (only transform)
X_val_scaled = X_val.copy()
X_val_scaled[numeric_columns_test] = scaler.transform(X_val[numeric_columns_test])

there's still some odd remaining floats  (that prevent me converting to Tensors), so deal with them here:

In [41]:
# dtypes of all columns
column_dtypes = X_train_scaled.dtypes

# only columns with dtype 'object'
object_columns = column_dtypes[column_dtypes == 'object'].index.tolist()

print("Columns with dtype 'object':")
print(object_columns)

print("\n values in each object column:")

for column in object_columns:
    print(f"\n Column: {column}")
    print(X_train_scaled[column].head(30))

Columns with dtype 'object':
['mergeid', 'hhid', 'coupleid', 'NUTS1_floods', 'NUTS2_floods']

 values in each object column:

 Column: mergeid
139010    DK-367119-03
258621    GR-863675-01
362486    PT-347341-02
59849     CZ-013379-01
34900     Bf-717782-02
166174    EE-509767-01
368040    RO-667384-02
100039    Cg-666351-01
222681    FI-933616-01
104707    Ci-963123-01
108745    DE-158453-01
104985    DE-007604-01
227162    FR-224531-01
11060     AT-521280-01
29305     Bf-355197-02
389388    SE-846670-02
256415    GR-758399-01
343861    NL-879907-02
222250    FI-794198-01
169784    EE-645194-01
7987      AT-383972-01
228749    FR-322665-01
186385    ES-330504-01
365128    PT-871084-02
281360    IT-225114-02
1451      AT-065734-01
13558     AT-636509-02
333933    NL-293121-02
389778    SE-864367-02
63407     CZ-152255-01
Name: mergeid, dtype: object

 Column: hhid
139010    DK-367119-A
258621    GR-863675-A
362486    PT-347341-A
59849     CZ-013379-A
34900     Bf-717782-A
166174    EE-

NB THIS WAS DIFFERENT WHEN I RAN IT IN THE NEURAL NET SCRIPT... SO WE'LL NEED TO ADAPT THE BELOW CODES...

In [42]:
# Process 'birth_country' and 'citizenship' by removing the numeric prefix
X_train_scaled['birth_country'] = X_train_scaled['birth_country'].str.split('. ').str[1]
X_train_scaled['citizenship'] = X_train_scaled['citizenship'].str.split('. ').str[1]

# Process 'NUTS2_floods' by replacing '.' or '0' with 'missing'
X_train_scaled['NUTS2_floods'] = X_train_scaled['NUTS2_floods'].replace(['.', '0'], 'missing')

print("Updated 'birth_country' column:")
print(X_train_scaled['birth_country'].head(10))

print("\nUpdated 'citizenship' column:")
print(X_train_scaled['citizenship'].head(10))

print("\nUpdated 'NUTS2_floods' column:")
print(X_train_scaled['NUTS2_floods'].head(10))

Updated 'birth_country' column:
139010        Denmark
258621         Greece
362486       Portugal
59849            Czec
34900         Belgium
166174        Estonia
368040        Romania
100039    Switzerland
222681        Finland
104707          Italy
Name: birth_country, dtype: object

Updated 'citizenship' column:
139010        Danish
258621         Greek
362486    Portuguese
59849          Czech
34900        Belgian
166174      Estonian
368040      Romanian
100039         Swiss
222681       Finnish
104707         Swiss
Name: citizenship, dtype: object

Updated 'NUTS2_floods' column:
139010    missing
258621    missing
362486       PT11
59849        CZ07
34900         BE3
166174    missing
368040    missing
100039    missing
222681    missing
104707    missing
Name: NUTS2_floods, dtype: object


Now X_train is sorted, still having issues with y_train

In [43]:
print(y_train.unique())

['1. Excellent', '4. Fair', '3. Good', '5. Poor', '2. Very good', '-12. don't know / refusal', '-15. no information']
Categories (7, object): ['-15. no information' < '-12. don't know / refusal' < '1. Excellent' < '2. Very good' < '3. Good' < '4. Fair' < '5. Poor']


In [44]:
y_train = y_train.str.extract(r'\d+\.\s*(.*)')[0].copy()

y_train = y_train.replace(np.nan, "missing").copy()

print(y_train.unique())

['Excellent' 'Fair' 'Good' 'Poor' 'Very good' "don't know / refusal"
 'no information']


1) the problematic columns are different in this script that the neural net script... we'll need to adapt it accordngly, but they provide a good template

2) we should move these processing steps to before the X / y, test/train/val split so we're acting on the original df which we then can just export out as a csv and load directly into our other scripts as we're working on them.